In [ ]:
import os
import openai
from dotenv import load_dotenv

# Get Configuration Settings
load_dotenv()

openai.api_type = 'azure'
openai.api_key = os.getenv('AOAI_KEY')
openai.api_base =  os.getenv('AOAI_ENDPOINT') 
openai.api_version = os.environ.get("AOAI_VERSION","2023-03-15-preview")  


In [ ]:
#https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
response = openai.ChatCompletion.create(
    engine="gpt-35-turbo", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
    messages=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": "What's the difference between garbanzo beans and chickpeas?"}
    ]
)

print(response)

print(response['choices'][0]['message']['content'])

In [7]:
conversation=[{"role": "system", "content": "You are a helpful assistant."}]

while(True):
    user_input = input()      
    if user_input == "exit" or user_input == "quit" or user_input == "":
        break
    conversation.append({"role": "user", "content": user_input})

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
        messages = conversation
    )

    conversation.append({"role": "assistant", "content": response['choices'][0]['message']['content']})
    print("\n" + response['choices'][0]['message']['content'] + "\n")


I'm sorry, but as an AI language model, I am incapable of predicting the future. However, the 2022 World Series has not yet occurred, as it is an upcoming event. Once the championship is held, I'm sure sports news outlets will be excited to report on the winner.



# Bot for reading content from a web site

In [ ]:
url=input("Enter a url for a web site")
conversation=[{"role": "system", "content": f"You are a helpful assistant that knows alot about {url}. Read the webside and answer questions."}]

while(True):
    user_input = input()      
    if user_input == "exit" or user_input == "quit" or user_input == "":
        break
    conversation.append({"role": "user", "content": user_input})

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
        messages = conversation
    )

    conversation.append({"role": "assistant", "content": response['choices'][0]['message']['content']})
    print("\n" + response['choices'][0]['message']['content'] + "\n")

In [ ]:
prompt="""The CSS code for a color like a blue sky at dusk:

background-color: #"""
response=openai.Completion.create(prompt=prompt,
    engine='text-davinci-003',
    max_tokens=64, 
    temperature=0, 
    top_p=1, 
    frequency_penalty=0.0, 
    presence_penalty=0.0, 
    stop=[";"])

print(f"{response.choices[0].text}")

In [ ]:
prompt="Write a tagline for an ice cream shop."
response=openai.Completion.create(
    prompt=prompt,
    engine='text-davinci-003'
)

print(f"{response.choices[0].text}")


In [ ]:
prompt="""
The following is a list of companies and the categories they fall into:

Apple, Facebook, Fedex

Apple
Category:
"""

response = openai.Completion.create(
  engine='text-davinci-003',
  prompt=prompt,
  temperature=0,
  max_tokens=64,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

print(f"{response.choices[0].text}")

# All about SQL Helper

In [9]:
prompt="""
### Postgres SQL tables, with their properties:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
#Create a SQL query for A query to list the names of the departments which employed more than 10 employees in the last 3 months
SELECT
"""

response = openai.Completion.create(
  engine='code-davinci-002',
  prompt=prompt,
  temperature=0,
  max_tokens=150,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
    stop=["#",";"]
)

print(f"{response.choices[0].text}")

    d.name
FROM
    Department d
    INNER JOIN Employee e ON d.id = e.department_id
    INNER JOIN Salary_Payments sp ON e.id = sp.employee_id
WHERE
    sp.date > NOW() - INTERVAL '3 months'
GROUP BY
    d.name
HAVING
    COUNT(e.id) > 10


## Bot to write SQL for multiple questions

In [22]:
# Same thing but will ONLY generate the SQL statement
prompt="""
Postgres SQL tables, with their properties:
Employee(id, name, department_id)
Department(id, name, address)
Salary_Payments(id, employee_id, amount, date)

#Create a SQL query for "{0}"
"""

while(True):
    user_input = input()      
    if user_input == "exit" or user_input == "quit" or user_input == "":
        break

    sql = prompt.format(user_input)

    response = openai.Completion.create(
    engine='code-davinci-002',
    prompt=sql,
    temperature=0,
    max_tokens=150,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
      stop=["#",";"]
    )

    print(f"{response.choices[0].text}")

SELECT COUNT(*) FROM Department

SELECT department.name, COUNT(employee.id)
FROM employee
INNER JOIN department ON employee.department_id = department.id
GROUP BY department.name



SELECT department.name, COUNT(employee.id) AS employee_count
FROM employee
JOIN department ON employee.department_id = department.id
GROUP BY department.name
ORDER BY employee_count DESC
LIMIT 3


## This using GPT model and gets more conversational
Do you like the conversation? You can easily parse the SQL returned with LangChain or other coding to then turnaround and execute the SQL for the end users

In [23]:
# Chat SQL with ChatGPT
prompt="""
Tables and columns:
Employee(id, name, department_id)
Department(id, name, address)
Salary_Payments(id, employee_id, amount, date)

You are a bot that generates SQL code based on the "Tables and columns". Return standard SQL queries that a user could then run in a tool like SQL Server Management Studio
"""

conversation=[{"role": "system", "content": prompt}]

while(True):
    user_input = input()      
    if user_input == "exit" or user_input == "quit" or user_input == "":
        break
    conversation.append({"role": "user", "content": user_input})

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
        messages = conversation
    )

    conversation.append({"role": "assistant", "content": response['choices'][0]['message']['content']})
    print("\n" + response['choices'][0]['message']['content'] + "\n")


I'm sorry, I didn't understand your request. Can you please provide more information on what you need?



In [ ]:
conversation=[{"role": "system", "content": "You are a helpful assistant."}]

while(True):
    user_input = input()      
    if user_input == "exit" or user_input == "quit" or user_input == "":
        break
    conversation.append({"role": "user", "content": user_input})

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
        messages = conversation
    )

    conversation.append({"role": "assistant", "content": response['choices'][0]['message']['content']})
    print("\n" + response['choices'][0]['message']['content'] + "\n")


I'm sorry, but as an AI language model, I am incapable of predicting the future. However, the 2022 World Series has not yet occurred, as it is an upcoming event. Once the championship is held, I'm sure sports news outlets will be excited to report on the winner.



In [ ]:
prompt="""
Convert movie titles into emoji.

Back to the Future: 👨👴🚗🕒 
Batman: 🤵🦇 
Transformers: 🚗🤖 
Star Wars:
"""

response = openai.Completion.create(
  engine='text-davinci-003',
  prompt=prompt,
  temperature=0.8,
  max_tokens=60,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["\n"]
)

print(f"{response.choices[0].text}")


In [ ]:
prompt="""
class Log:
    def __init__(self, path):
        dirname = os.path.dirname(path)
        os.makedirs(dirname, exist_ok=True)
        f = open(path, "a+")

        # Check that the file is newline-terminated
        size = os.path.getsize(path)
        if size > 0:
            f.seek(size - 1)
            end = f.read(1)
            if end != "\n":
                f.write("\n")
        self.f = f
        self.path = path

    def log(self, event):
        event["_event_id"] = str(uuid.uuid4())
        json.dump(event, self.f)
        self.f.write("\n")

    def state(self):
        state = {"complete": set(), "last": None}
        for line in open(self.path):
            event = json.loads(line)
            if event["type"] == "submit" and event["success"]:
                state["complete"].add(event["id"])
                state["last"] = event
        return state

#
Here's what the above class is doing:
1.
"""

response = openai.Completion.create(
  engine='code-davinci-002',
  prompt=prompt,
  temperature=0,
  max_tokens=64,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["#"]
)

print(f"{response.choices[0].text}")

In [ ]:
prompt="""
Q: Who is Batman?
A: Batman is a fictional comic book character.

Q: What is torsalplexity?
A: ?

Q: What is Devz9?
A: ?

Q: Who is George Lucas?
A: George Lucas is American film director and producer famous for creating Star Wars.

Q: What is the capital of California?
A: Sacramento.

Q: What orbits the Earth?
A: The Moon.

Q: Who is Fred Rickerson?
A: ?

Q: What is an atom?
A: An atom is a tiny particle that makes up everything.

Q: Who is Alvan Muntz?
A: ?

Q: What is Kozar-09?
A: ?

Q: How many moons does Mars have?
A: Two, Phobos and Deimos.

Q: What's a language model?
A:
"""

response = openai.Completion.create(
  engine='text-davinci-003',
  prompt=prompt,
  temperature=0,
  max_tokens=60,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

print(f"{response.choices[0].text}")

In [ ]:
prompt="""
##### Fix bugs in the below function
 
### Buggy Python
import Random
a = random.randint(1,12)
b = random.randint(1,12)
for i in range(10):
    question = "What is "+a+" x "+b+"? "
    answer = input(question)
    if answer = a*b
        print (Well done!)
    else:
        print("No.")
    
### Fixed Python
"""

response = openai.Completion.create(
  engine='code-davinci-002',
  prompt=prompt,
  temperature=0,
  max_tokens=182,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["###"]
)

print(f"{response.choices[0].text}")

In [ ]:
prompt="""
You: How do I combine arrays?
JavaScript chatbot: You can use the concat() method.
You: How do you make an alert appear after 10 seconds?
JavaScript chatbot
"""

response = openai.Completion.create(
  engine='code-davinci-002',
  prompt=prompt,
  temperature=0,
  max_tokens=60,
  top_p=1.0,
  frequency_penalty=0.5,
  presence_penalty=0.0,
  stop=["You:"]
)

print(f"{response.choices[0].text}")

In [ ]:
prompt="""
Extract the airport codes from this text:

Text: "I want to fly from Los Angeles to Miami."
Airport codes: LAX, MIA

Text: "I want to fly from Orlando to Boston"
Airport codes:
"""

response = openai.Completion.create(
  engine='text-davinci-003',
  prompt=prompt,
  max_tokens=60,
  temperature=0,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["\n"]
)

print(f"{response.choices[0].text}")

In [ ]:
prompt="""
You: What have you been up to?
Friend: Watching old movies.
You: Did you watch anything interesting?
Friend:
"""

response = openai.Completion.create(
    engine='text-davinci-003',
    prompt=prompt,
    temperature=0.5,
    max_tokens=60,
    top_p=1.0,
    frequency_penalty=0.5,
    presence_penalty=0.0,
    stop=["You:"]
)

print(f"{response.choices[0].text}")

In [ ]:
prompt="""
Brainstorm some ideas combining VR and fitness:
"""

response = openai.Completion.create(
    engine='text-davinci-003',
    prompt=prompt,
    temperature=0.6,
    max_tokens=150,
    top_p=1.0,
    frequency_penalty=1,
    presence_penalty=1
)

print(f"{response.choices[0].text}")

In [ ]:
prompt="""
Create a list of 8 questions for my interview with a science fiction author:
"""

response = openai.Completion.create(
    engine='text-davinci-003',
    prompt=prompt,
    temperature=0.5,
    max_tokens=150,
    top_p=1.0,
    frequency_penalty=0,
    presence_penalty=0
)

print(f"{response.choices[0].text}")

In [ ]:
prompt="""
Create an outline for an essay about Nikola Tesla and his contributions to technology:
"""

response = openai.Completion.create(
    engine='text-davinci-003',
    prompt=prompt,
    temperature=0.3,
    max_tokens=150,
    top_p=1.0,
    frequency_penalty=0,
    presence_penalty=0
)

print(f"{response.choices[0].text}")

In [ ]:
prompt="""
What are 5 key points I should know when studying Ancient Rome?
"""

response = openai.Completion.create(
    engine='text-davinci-003',
    prompt=prompt,
    temperature=0.3,
    max_tokens=150,
    top_p=1.0,
    frequency_penalty=0,
    presence_penalty=0
)

print(f"{response.choices[0].text}")